# Mapping Reads to a Reference Genome
<!---
** Add Intro Text***
-->

## Shell Variables

In [ ]:
# Source the config script
source bioinf_intro_config.sh

ls $CUROUT

## Mapping with STAR


In [ ]:
STAR \
    --runMode alignReads \
    --twopassMode None \
    --genomeDir $GENOME_DIR \
    --readFilesIn $TRIMMED/27_MA_P_S38_L002_R1_001.trim.fastq.gz \
    --readFilesCommand gunzip -c \
    --outFileNamePrefix ${STAR_OUT}/27_MA_P_S38_L002_R1_ \
    --quantMode GeneCounts \
    --outSAMtype BAM Unsorted \
    --outSAMunmapped Within

We will start with these parameters, but there is an extensive list of command line options detailed in the [STAR Manual](https://github.com/alexdobin/STAR/blob/master/doc/STARmanual.pdf), it is a good idea to read through and try to understand all of them.  We will discuss some more later.

* --runMode alignReads : map reads 
* --twopassMode : Run one pass or two? If two-pass mode is on, STAR tries to discover novel junctions, then reruns mapping with these added to the annotation
* --genomeDir : directory containing the genome index
* --readFilesIn : input FASTQ
* --readFilesCommand gunzip -c : use "gunzip -c" to uncompress FASTQ on-the-fly, since it is gzipped
* --outFileNamePrefix : prefix (and path) to use for all output files
* --quantMode GeneCounts : output a table of read counts per gene
* --outSAMtype BAM Unsorted : output an unsort BAM file
* --outSAMunmapped Within : included unmapped reads in the BAM file

### STAR Output
So what happened? Let's take a look . . .

In [ ]:
ls ${STAR_OUT}

In [ ]:
head ${STAR_OUT}/27_MA_P_S38_L002_R1*

STAR generates several files for each FASTQ:
* Log.out : lots of details of the run, including all parameters used
* Log.final.out : Important summary statistics
* ReadsPerGene.out.tab : Count table, the main thing we are interested in
* SJ.out.tab : All splice junctions, including ones from the GTF and novel junctions discovered by STAR
* Log.progress.out: run statistics updated during run, not so interesting at the end

Let's take a closer look at Log.final.out

In [ ]:
cat ${STAR_OUT}/27_MA_P_S38_L002_R1_Log.final.out

#### Sanity Check: Number of Reads
Are the number of input reads what we expect? Let's look at how many reads are in the input FASTQ

In [ ]:
zcat $TRIMMED/27_MA_P_S38_L002_R1_001.trim.fastq.gz | awk '{s++}END{print s/4}' 

#### Sanity Check: Unmapped Reads
It is always a good idea to examine a sample of unmapped reads to figure out what they are. The easiest way to do this is with BLAST.  In the past I have discovered that an experiment was contaminated with a different species by BLASTing unmapped reads.  In that case there were a large number of unmapped reads, which raised my suspicions.

Even with a high rate of mapped reads, it is worth spending a few minutes to check them out.  The simplest thing to do is use `samtools` to generate a FASTA from the unmapped.bam, grab a few of these sequences, and then [BLAST them against the nr database](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastn&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome)

In [ ]:
samtools fasta -f 4 ${STAR_OUT}/27_MA_P_S38_L002_R1_Aligned.out.bam | head -n20